In [1]:
import os
import sys
from pathlib import Path
if Path(os.getcwd()).name == "notebooks":
    os.chdir("../")
    print(f"Working dir changed to {os.getcwd()}")
sys.path.append("src")

Working dir changed to d:\my_documents\personal_projects\FPL


In [2]:
import sqlite3
import yaml
import pandas as pd
import re
from thefuzz import process

In [3]:
def load_yaml_as_dict(yaml_file):
    with open(yaml_file) as stream:
        return yaml.safe_load(stream)
data_params = load_yaml_as_dict("conf/base/parameters/data.yml")
model_params = load_yaml_as_dict("conf/base/parameters/model.yml")

FileNotFoundError: [Errno 2] No such file or directory: 'conf/base/parameters/data.yml'

In [4]:
def select_relevant_rows(processed_data):
    ma_cols = processed_data.filter(regex=r"\w+_ma\d+").columns
    ma_cols = set([re.sub(r"_ma\d+", "", col) for col in ma_cols])
    excluded = [
        "value",
        "att_total",
        "home_att_total",
        "away_att_total",
        "def_total",
        "home_def_total",
        "away_def_total",
        "fpl_points",
    ]
    ma_cols = [col for col in ma_cols if col not in excluded]
    non_features = ["cached", "start", "match_points", "league_points"]
    non_features += processed_data.filter(regex="_elo$").columns.tolist()
    non_features += processed_data.filter(regex="_opp$").columns.tolist()
    to_drop = [col for col in ma_cols + non_features if col in processed_data.columns]
    processed_data = processed_data.drop(to_drop, axis=1)
    return processed_data

In [5]:
conn = sqlite3.connect("data/fpl.db")

fpl_data = pd.read_sql(f"select full_name, season, sum(total_points) from raw_fpl_data group by full_name, season", conn)

conn.close()

In [18]:
player_name_mapping = pd.read_csv("data/preprocess/player_name_mapping.csv")

search_for = "Olayinka Fredrick Oladotun Ladapo"
matched = process.extract(
    search_for, player_name_mapping["fpl_name"].tolist(), limit=10, 
)
top_matches = [m for m in matched if m[1] > 60]
if not top_matches:
    top_matches = matched

for fpl_name, score in top_matches:
    data = fpl_data.query(f"full_name == '{fpl_name}'")
    data = pd.merge(player_name_mapping[["fbref_name", "fpl_name"]], data, how="right", right_on="full_name", left_on="fpl_name").drop("full_name", axis=1)
    display(data)

,fbref_name,fpl_name,season,sum(total_points)
0,NaN,Olayinka Fredrick Oladotun Ladapo,2016-2017,0.0
1,NaN,Olayinka Fredrick Oladotun Ladapo,2017-2018,1.0


,fbref_name,fpl_name,season,sum(total_points)
0,Freddie Ladapo,Freddie Ladapo,2024-2025,NaN


,fbref_name,fpl_name,season,sum(total_points)
0,NaN,Benjamin Fredrick,2023-2024,0.0


,fbref_name,fpl_name,season,sum(total_points)
0,Ola Aina,Ola Aina,2024-2025,NaN


,fbref_name,fpl_name,season,sum(total_points)
0,NaN,Olu Aina,2016-2017,3.0
1,NaN,Olu Aina,2020-2021,102.0
2,NaN,Olu Aina,2023-2024,45.0


,fbref_name,fpl_name,season,sum(total_points)
0,NaN,Fredrik Ulvestad,2016-2017,0.0
1,NaN,Fredrik Ulvestad,2017-2018,0.0


,fbref_name,fpl_name,season,sum(total_points)
0,Ryan Fredericks,Ryan Fredericks,2018-2019,38.0
1,Ryan Fredericks,Ryan Fredericks,2019-2020,74.0
2,Ryan Fredericks,Ryan Fredericks,2020-2021,28.0
3,Ryan Fredericks,Ryan Fredericks,2021-2022,7.0
4,Ryan Fredericks,Ryan Fredericks,2022-2023,12.0
5,Ryan Fredericks,Ryan Fredericks,2023-2024,0.0


,fbref_name,fpl_name,season,sum(total_points)
0,NaN,Frederick Woodman,2018-2019,0.0
1,NaN,Frederick Woodman,2019-2020,0.0


,fbref_name,fpl_name,season,sum(total_points)
0,Érik Lamela,Erik Lamela,2016-2017,28.0
1,Érik Lamela,Erik Lamela,2017-2018,55.0
2,Érik Lamela,Erik Lamela,2018-2019,67.0
3,Érik Lamela,Erik Lamela,2019-2020,65.0
4,Érik Lamela,Erik Lamela,2020-2021,28.0
5,Érik Lamela,Erik Lamela,2021-2022,0.0


In [74]:
matched = process.extract(
    search_for, player_name_mapping["fbref_name"].tolist(), limit=10, 
)
matched

[('Diego Costa', 86),
 ('Kevin De Bruyne', 86),
 ('David de Gea', 86),
 ('Marten de Roon', 86),
 ('Hélder Costa', 86),
 ('Donny van de Beek', 86),
 ('Micky van de Ven', 86),
 ('Ederson', 64),
 ('Eric Dier', 60),
 ('Oscar', 60)]

In [75]:
player_name_mapping.query("fbref_name == 'José Izquierdo'")

,fbref_id,fbref_name,fpl_name,fuzzy_score,total_points,review,duplicated,missing_matchlogs
596,ca2bc3df,José Izquierdo,José Heriberto Izquierdo Mena,100.0,NaN,NaN,NaN,NaN
